# Web scrapping


## Flujo de trabajo:

- Pedir la página web al servidor. 
- Parsearla; identificar y extraer los elementos que nos interesan. 
- Guardar la información extraída. 

## Herramientas a utilizar:
Para hacer webscrapping en python, necesitamos las siguientes herramientas: 

1. Para pedir información:
    - requests
2. Para parsear la información
    - BeautifulSoup
3. Para lo demás:  
    - pandas

## Requests

Vital para interactuar con objetos html (como en APIS).

### Headers:
los headers son clave, pues simularán parámetros para hacer nuestra búsqueda (e.g. Tipo de navegador, versión, dispositivo, caching, redirecciones, sesiones, etc). Puedes encontrar más [aquí]()

In [2]:
import requests 
import json
from bs4 import BeautifulSoup
import re

In [3]:
URL = 'http://httpbin.org/headers'
resp = requests.get(URL)

print('Respuesta sin headers:')
print(resp.text)

Respuesta sin headers:
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-670e56c7-419b91fb09fbbd6517c113f2"
  }
}



In [4]:

print('Respuesta con headers:')
posibles_headers =[
{
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
},
{
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/537.36'
},
{
  'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0'  
}
]
    
resp_con_headers = requests.get(URL, headers = posibles_headers[2])
print(resp_con_headers.text)

Respuesta con headers:
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0", 
    "X-Amzn-Trace-Id": "Root=1-670e56c8-12a00bef0bc9624472a31e64"
  }
}



In [5]:
# Pedimos info a wikipedia
url = 'https://es.wikipedia.org/wiki/Wikipedia:Portada'

# Guardamos el objeto que nos devuelve
respuesta = requests.get(url)

print(f'Tipo de Objeto: {type(respuesta)} \n')
print(f'Código de estado: {respuesta.status_code} \n')
print(f'Data: {respuesta.text} \n')

Tipo de Objeto: <class 'requests.models.Response'> 

Código de estado: 200 

Data: <!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-disabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-not-available" lang="es" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Wikipedia, la enciclopedia libre</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-



###  Detour: Utilizando requests para otros fines:
Requests nos puede ayudar a descargar todo tipo de contenidos, como imágenes, videos, documentos. En el ejemplo descargamos una presentación 

In [6]:
ppt = requests.get("https://github.com/ccsuehara/varios/raw/refs/heads/main/presentacion.pptx")

with open('ejemplo.pptx', 'wb') as file:
    for chunk in ppt.iter_content(chunk_size=1024): ## bajando información por chunks, en vez de todo junto 
        file.write(chunk)

### Sigamos: 

## Navegando por el DOM como si fuera una Sopa de etiquetas.
 !["beautiful soup"](webscrapdom.png)


## Etiquetas más importantes a buscar

- `div` - secciones grandes.  
- `table` - tablas, divididas en   `tr` (filas) and `td` (datum).  
- `form` - etiqueta input para ser enviada.  
- `ul`/`ol` -listas (no ordenadas y ordenadas), contiene `li` (items de listas).  

Atributos importantes: 
- `id` y `class`
Los tags se llaman diferente aquí; los web developers utilizan estos tags para diseño e interacción. 
- `ids` son únicos; `classes` son grupos

In [7]:
url = "https://www.scrapethissite.com/pages/simple/"

In [8]:
## 1. Obtenemos el HTML
respuesta  = requests.get(url)
## 2. Volvemos HTML a string. 
html_texto = respuesta.text
## 3. Parseando el HTML
soup = BeautifulSoup(html_texto, "html.parser") ## puede ser "lxml" o "html5lib"

### El método `find`
Sólo el primer elemento

In [9]:
soup.find("h3").text

'\n\n                            Andorra\n                        '

### El método `find_all`

In [74]:
all_h3 = soup.find_all("h3") ## nos  da todoslos h3

In [75]:
# Utilizando el get_text
for seccion in all_h3:
  print(seccion.get_text(strip=True))

Andorra
United Arab Emirates
Afghanistan
Antigua and Barbuda
Anguilla
Albania
Armenia
Angola
Antarctica
Argentina
American Samoa
Austria
Australia
Aruba
Åland
Azerbaijan
Bosnia and Herzegovina
Barbados
Bangladesh
Belgium
Burkina Faso
Bulgaria
Bahrain
Burundi
Benin
Saint Barthélemy
Bermuda
Brunei
Bolivia
Bonaire
Brazil
Bahamas
Bhutan
Bouvet Island
Botswana
Belarus
Belize
Canada
Cocos [Keeling] Islands
Democratic Republic of the Congo
Central African Republic
Republic of the Congo
Switzerland
Ivory Coast
Cook Islands
Chile
Cameroon
China
Colombia
Costa Rica
Cuba
Cape Verde
Curacao
Christmas Island
Cyprus
Czech Republic
Germany
Djibouti
Denmark
Dominica
Dominican Republic
Algeria
Ecuador
Estonia
Egypt
Western Sahara
Eritrea
Spain
Ethiopia
Finland
Fiji
Falkland Islands
Micronesia
Faroe Islands
France
Gabon
United Kingdom
Grenada
Georgia
French Guiana
Guernsey
Ghana
Gibraltar
Greenland
Gambia
Guinea
Guadeloupe
Equatorial Guinea
Greece
South Georgia and the South Sandwich Islands
Guatemala
G

In [76]:
## Obteniendo lo mismo pero con nombre de las clases
# Clase
h3_class = soup.find_all('h3', class_  = "country-name")

In [77]:
# Utilizando el get_text
for seccion in h3_class:
  print(seccion.get_text(strip=True))

Andorra
United Arab Emirates
Afghanistan
Antigua and Barbuda
Anguilla
Albania
Armenia
Angola
Antarctica
Argentina
American Samoa
Austria
Australia
Aruba
Åland
Azerbaijan
Bosnia and Herzegovina
Barbados
Bangladesh
Belgium
Burkina Faso
Bulgaria
Bahrain
Burundi
Benin
Saint Barthélemy
Bermuda
Brunei
Bolivia
Bonaire
Brazil
Bahamas
Bhutan
Bouvet Island
Botswana
Belarus
Belize
Canada
Cocos [Keeling] Islands
Democratic Republic of the Congo
Central African Republic
Republic of the Congo
Switzerland
Ivory Coast
Cook Islands
Chile
Cameroon
China
Colombia
Costa Rica
Cuba
Cape Verde
Curacao
Christmas Island
Cyprus
Czech Republic
Germany
Djibouti
Denmark
Dominica
Dominican Republic
Algeria
Ecuador
Estonia
Egypt
Western Sahara
Eritrea
Spain
Ethiopia
Finland
Fiji
Falkland Islands
Micronesia
Faroe Islands
France
Gabon
United Kingdom
Grenada
Georgia
French Guiana
Guernsey
Ghana
Gibraltar
Greenland
Gambia
Guinea
Guadeloupe
Equatorial Guinea
Greece
South Georgia and the South Sandwich Islands
Guatemala
G

In [78]:
## almacenando la información
paises = [pais.get_text(strip=True) for pais in h3_class]


In [81]:
## Obteniendo poblacion. 
h3_population = soup.find_all("span", class_ = "country-population")
poblacion = [pop.get_text(strip=True) for pop in h3_population]


## Ejercicios:
- Realiza el mismo ejercicio para hallar la capital y el area en km2
- Guarda el resultado en un dataframe. 

### Web scrap en 3 líneas: 

In [17]:
# Buscaremos titulares en portada de el Comercio. 
r = requests.get('https://elcomercio.pe/', headers = posibles_headers[1])
soup = BeautifulSoup(r.text)
tags = soup.find_all('h2', class_='fs-wi__title')

## Ejercicio

- Identifica 2 webs de noticias que te interesen. 
- Encuentra la etiqueta que refieren a los titulares de portada de cada una. 
- Extrae los titulares con técnicas de webscrapping. 
- Guárdalo en un dataframe. Tus columnas pueden ser:
    - nombre del periódico
    - fecha y hora de la obtencion
    - titulos  


 